# Count Based Distributional Semantics

In this notebook we will implement a simple count based model. We will not try to optimize the model in order to get the best results possible. Instead we will try to keep things as simple as possible. Thus the main principles can become clear. 

We will use a mid-sized corpus as a compromise between fast processing and usefull results. 

In this notebook we will use German data. However, not much knowledge of German is required. And learning some new German words on the fly is not too bad.

**<span style="color:red">Caution</span>**

Some of the cells require quite long computation time!

## Data

We use a corpus of 300k sentences from wikipedia collected by the university of Leipzig. You can download the required data from http://wortschatz.uni-leipzig.de/en/download/ . The file used here is **deu_wikipedia_2016_300K-sentences**

### Einlesen der Daten

The texts in the corpus are already split into sentences. We read thes sentence and word-tokenize each sentence. To save time we use the infected words and do not do anly lemmatization or stemming.

In [1]:
import codecs
import nltk

sentences = []

#Caution: change the path to this file!
quelle = codecs.open('../../TMNB/Corpora/deu_wikipedia_2016_300K/deu_wikipedia_2016_300K-sentences.txt','r','utf8')
for line in quelle:
    nr,satz = line.split('\t')
    sentences.append(nltk.word_tokenize(satz.strip()))

Let us check whether the senteces are in the list as we expect them to be:

In [2]:
sentences[447]

['1819',
 'wurde',
 'daher',
 'ein',
 'neues',
 '„',
 'Vereidigungsbuch',
 '“',
 'angelegt',
 '.']

## Vectors of Co-occurrence Values


We write a function that computes vectors with co-occurrence numbers for a number of  words with a given list of 'context' words. As context words we take all words that exceed a specified minimum frequency. KCo-occurrence with rare words will hardly contribute to the comparison between context vectors. Another parameter that needs to be set is the maximum distance between two words to count as co-competition, known as the window size. We proceed sentence by sentence here. This means that the last word of a sentence and the first word of the next sentence are never counted as co-occurring. Note, however, that sentence bondaries are often not taken into account in such procedures. It is not clear whether this has a seriosu impact. Another detail to note is that we first calculate the window size for the competition, and then determine the relevant words in the window. Alternatively, you can first remove all irrelevant words and then determine the words within the window.

In general, a larger window can compensate for a too small corpus. In addition, a smaller window captures more syntactical properties of a word, while a larger window takes into account broader semantic relationships.

When all co-occurrence values are calculated, we normalize the length of the vectors.

In [1]:
from scipy import sparse
from collections import Counter
import numpy as np

def mag(x):
    return np.sqrt(x.dot(x))


def makeCV(words,sentences, window = 2, minfreq = 10):
    #first count all words
    freq = Counter()
    for s in sentences:
        freq.update(s)
        
    #determine which words have to be used as features or context words
    context_words = [w for w,f in freq.items() if f > minfreq]
    
    #we add all words to the context words, if they are not already in that list. Just for convenience to have all words in one list. 
    for w in words:
        if w not in context_words:
            context_words.append(w)
    dim = len(context_words)
    
    #Give each context word a unique number and build dictionaries to switch between numbers and words
    cw2nr = {}
    for i in range(dim):
        cw = context_words[i]
        cw2nr[cw] = i
              
    w2nr = {}
    for i in range(len(words)):
        w = words[i]
        w2nr[w] = i
    
    
    #initialize a matrix
    #We use  a sparse matrix class from scipy
    matrix = sparse.lil_matrix((len(words),dim))
    
    #Now we start the real work. We iterate through all sentences and cont the co-occurrences!
    n = 0
    for s in sentences:
        n+=1
        i_s = [cw2nr.get(w,-1) for w in s]
        for i in range(len(s)):
            w = s[i]
            if w in words:
                i_w = w2nr[w]
                for j in range(max(0,i-window),min(i+window+1,len(s))):
                    if i != j: # a word is not in its own context!
                        i_cw = i_s[j]
                        if i_cw > 0:
                            matrix[i_w,i_cw] += 1
          
    #finally make a dictionary with vectors for each word
    wordvectors = {}
    for w,i_w in w2nr.items():
        v = matrix[i_w].toarray()[0]
        wordvectors[w] = v/mag(v)
    return wordvectors

ModuleNotFoundError: No module named 'scipy'

Let us test the function for a short list of words:

In [4]:
vectors_count =  makeCV(['Kloster','Kirche','Garten','Haus','Hof','Schweden','Deutschland','betrachten','anschauen','beobachten'],sentences,window=2)

Since the vectors have the same length, we can use the inner product, which is identical to the cosine, for comparison:

In [5]:
print(vectors_count['Schweden'].dot(vectors_count['Deutschland']))
print(vectors_count['Schweden'].dot(vectors_count['Hof']))

0.8827709543096601
0.702106425947288


Next we want to know, what words are most similar to a given word. To do so, we need to compare a word with each other word in the list. We use an ordered list to store the results. Since these list always sort ascending, we need to consider always the last elements of this list. Finallye, we return the results in inverse order.

In [6]:
import bisect 

def most_similar(word,vectors,n):
    best = []
    vec_w = vectors[word]
    for z in vectors:
        if z == word:
            continue
        sim = vec_w.dot(vectors[z])

        #we have to add this result only, if we do not yet have n results, or if the similarity is larger than the similarity with the last element in the list (actually the first since we sort ascending)
        if len(best) < n or sim > best[0][0]:
            bisect.insort(best,(sim,z))
            best = best[-n:]
       
    return best[::-1] #present the list in descending order

In [7]:
most_similar('Garten',vectors_count,3)

[(0.8130680728909148, 'Hof'),
 (0.664617096252052, 'Haus'),
 (0.6388171089882518, 'Schweden')]

It is more interesing to find the most similar word if we have more words to choose from. Let us collect some mid-frequency words to do so.

In [8]:
wortfrequenz = Counter()

for satz in sentences:
    wortfrequenz.update(satz)

vocabulary  = [w for w,f in wortfrequenz.items() if 30 < f < 3000]
vocabulary_size = len(vocabulary)
print(vocabulary_size)

12165


Now it takes some time to compute all vectors.

In [9]:
vectors_count = makeCV(vocabulary,sentences,window=2)

In [10]:
most_similar('Garten',vectors_count,10)

[(0.930302724001756, 'Park'),
 (0.8713290722481782, 'Saal'),
 (0.8648406935911824, 'Tempel'),
 (0.8631281311361956, 'Bezirk'),
 (0.8620724034396585, 'Sturm'),
 (0.858317906806122, 'Keller'),
 (0.8572571532271944, 'Raum'),
 (0.8549354330124461, 'Wald'),
 (0.8549228454240734, 'Kreis'),
 (0.846418171910536, 'Sinn')]

In [11]:
most_similar('betrachten',vectors_count,10)

[(0.9585602463943913, 'verstehen'),
 (0.9577321881251438, 'ermitteln'),
 (0.9575996519992644, 'schaffen'),
 (0.9566353415343977, 'bauen'),
 (0.9549111806749297, 'beobachten'),
 (0.9537790612894577, 'verwenden'),
 (0.9526877411161037, 'erhöhen'),
 (0.9525533613296538, 'bringen'),
 (0.9478641686123344, 'retten'),
 (0.947498120959553, 'kämpfen')]

In [12]:
most_similar('Schweden',vectors_count,10)

[(0.9462175999010392, 'Ungarn'),
 (0.9379372993961502, 'Polen'),
 (0.9353772897649918, 'Frankreich'),
 (0.9310094388128083, 'Australien'),
 (0.93014008593448, 'Spanien'),
 (0.9239015030110829, 'Russland'),
 (0.9229256210189929, 'Italien'),
 (0.922348217698481, 'England'),
 (0.9189920519494904, 'Ägypten'),
 (0.9183081684907252, 'Kanada')]

In [13]:
most_similar('Direktor',vectors_count,10)

[(0.9482467477877202, 'Leiter'),
 (0.9404963751501492, 'Chef'),
 (0.9368719576058703, 'Kommandeur'),
 (0.9341183267714239, 'Vorsitzender'),
 (0.923324253533616, 'Präsident'),
 (0.9104529525364085, 'Mitglied'),
 (0.8922921792779437, 'Vizepräsident'),
 (0.8728248376457493, 'Vorstandsmitglied'),
 (0.8697536816828365, 'Sekretär'),
 (0.8598231918685795, 'Kommandant')]

## Evaluation


The examples look nice, but how good are our vectors? There are a number of tests that can be done to check the quality of the vectors. One type of test is to compare the calculated similarity between two words to the similarity that subjects have given for word pairs. We can simply use the correlation to check the similarity.

A dataset with similarity assessments for German word pairs is the so-called Gur350 dataset, which was developed by the working group of Prof. Dr. Iryna Gurevych at the TU Darmstadt. Further information and a link for the download can be found here: https://www.informatik.tu-darmstadt.de/ukp/research_6/data/semantic_relatedness/german_relatedness_datasets/index.en.jsp

The data does not give a similarity between the words but a relationship, which is not exactly the same. For the evaluation, we only use the word pairs for which both words are contained in our vocabulary.

In [14]:
import math
testfile = codecs.open('../../TMNB/Corpora/wortpaare350.gold.pos.txt','r','utf8')
testfile.readline()

testdata = []
missing = set()
for line in testfile:
    w1,w2,sim,p1,p2 = line.split(':')
    if w1 in vocabulary and w2 in vocabulary:
        testdata.append((w1,w2,float(sim)))
    
def evaluate(data,vectors):
    gold = []
    predicted = []
    for v,w,sim in data:
        pred = vectors[v].dot(vectors[w])

        gold.append(sim)
        predicted.append(pred)
        #print(v,w,pred,sim,sep = '\t')
        
    
    av_p = sum(predicted)/len(predicted)
    av_g = sum(gold)/len(gold)
    
    cov = 0
    var_g = 0
    var_p = 0
    for s,t in zip(gold,predicted):
        cov += (s-av_g) * (t-av_p)
        var_g += (s-av_g) * (s-av_g)
        var_p += (t-av_p) * (t-av_p)
        
    return cov / math.sqrt(var_g*var_p)

In [15]:
len(testdata)

119

Note, that we are using only a very small part of the test data from the Gur350 dataset for testing. Thus we cannot compare our results to official results for these data! We can of course include more words in our vocabulary of mid-frequency words, but many words simpy are not in our corpus. 

In [16]:
evaluate(testdata,vectors_count)

0.16596262818469823

Despite the good looking lists that we generated above, we see that the calculated similarities correlate only very weakly with the similarity judgments.

## Somewhat more advanced


Unser erster Versuch bietet noch verschiedene Optimierungsmöglichkeiten. Zunächst nutzen wir nicht die einfache Kookkurrenzhäufigkeiten sondern die _Positive Pointwise Mutual Information_ (PPMI). Die  _Pointwise Mutual Information_ zwischen zwei Wörter a und b ist im Prinzip das Verhältnis zwischen der tatsächlichen Wahrscheinlichkeit, dass sie zusammen vorkommen, und die erwartete Wahrscheinlicheit, dass sie zusammen vorkommen, wenn ihre Vorkommen unabhängig voneinander wären. Wir definieren  $pmi(a,b) = log(\frac{p(ab)}{p(a)p(b)})$. Die ppmi(a,b) ist nun die pmi(a,b), wenn dieser Wert positiv ist, und sonst 0. Die Benutzung der ppmi beruht auf der Annahme, dass nur die Tatsache, dass Wörter häufiger als erwartet zusammen vorkommen interessant ist, während geringere Wahrscheinlichkeiten vermutlich eher auf Zufall beruhen oder jedenfalls nichts interesantes über Wortpaare aussagen.

Ein Problem unseres naiven Ansatzes war, dass die Vektoren extrem lang sind. Die Vektoren brauchen nicht nur sehr viel Speicherplatz. Viele der Dimensionen enthalten auch wenig oder nur redundante Informationen. Dieses Problem kann durch die Anwendung eines gängigen Dimensionsreduktionsverfahren gelöst werden. Hierunten werden wir hierfür Singular Value Decomposition nutzen und anschließend die wichtigste 100 Dimensionen nutzen.

Our first attempt still offers various possibilities for optimization. First of all we could not use the simple co-occurrence frequencies but the _Positive Pointwise Mutual Information_ (PPMI). The _Pointwise Mutual Information_ between two words a and b is in principle the ratio between the actual probability that they will occur together and the expected probability that they will occur if their occurrences were independent. We define $ pmi (a, b) = log (\ frac {p (ab)} {p (a) p (b)}) $. The ppmi(a, b) is now the pmi(a, b) if this value is positive, and otherwise 0. The use of the ppmi is based on the assumption that only the fact that words occur together more often than expected is interesting, while lower probabilities are more likely to be based on coincidence or in any case do not say anything interesting about word pairs.

Another problem with our naive approach was that the vectors are extremely long. The vectors not only need a lot of storage space. Many of the dimensions also contain little or only redundant information. This problem can be solved by using a common dimension reduction process. Below we will use Singular Value Decomposition for this and then use the most important 100 dimensions.

In [17]:
from sklearn.decomposition import TruncatedSVD
import math
import numpy as np

def makeCV_SVD(words,sentences, window = 2, minfreq = 10, size =256):

    freq = Counter()
    for s in sentences:
        freq.update(s)
    
    context_words = [w for w,f in freq.items() if f > minfreq]
    for w in words:
        if w not in context_words:
            context_words.append(w)
    dim = len(context_words)
    cw2nr = {}
    for i in range(dim):
        cw = context_words[i]
        cw2nr[cw] = i
              
    w2nr = {}
    for i in range(len(words)):
        w = words[i]
        w2nr[w] = i
    
    
    matrix = sparse.lil_matrix((len(words),dim))
    
    n = 0
    for s in sentences:
        n+=1
        i_s = [cw2nr.get(w,-1) for w in s]
        for i in range(len(s)):
            w = s[i]
            if w in words:
                i_w = w2nr[w]
                for j in range(max(0,i-window),min(i+window+1,len(s))):
                    if i != j:
                        i_cw = i_s[j]
                        if i_cw > 0:
                            matrix[i_w,i_cw] += 1
     
    #up to here nothing new
    N = matrix.sum()
    
    #Let us  get the probabilities for each word:
    freq_w = matrix.sum(axis = 1)
    freq_w = np.array(freq_w.T)[0]
    prob_w = np.array(freq_w) / N
    
    #Let us  get the probabilities for each context word:
    freq_cw = matrix.sum(axis = 0)
    freq_cw = np.array(freq_cw)[0]
    prob_cw = np.array(freq_cw) / N
     
    (rows,cols) = matrix.nonzero() #Returns a tuple of arrays (row,col) containing the indices of the non-zero elements of the matrix.
    for i_w,i_cw in zip(rows,cols):
        p = matrix[i_w,i_cw]/N
        p_w = prob_w[i_w]
        p_cw = prob_cw[i_cw]
        ppmi = max(0,math.log(p/(p_w * p_cw) )) 
        matrix[i_w,i_cw] = ppmi


    #We cannot be completely sure, that for every word we found at least som positive pmi-values.
    #The frequent neighbors of a word eventaully are not in the set of context words
    #A row with only zeros, will cause a problem for the SVD
    for i in range(len(words)):
        if np.sum(matrix[i]) == 0:
            print("Empty row for:",words[i])
            print("Please remove this word from the wordlist.")
        
    svd = TruncatedSVD(n_components=size)
    svd.fit(matrix)
    matrix = svd.transform(matrix)
    wordvectors = {}
    for w,i_w in w2nr.items():
        v = matrix[i_w] 
        wordvectors[w] = v/mag(v)
    return wordvectors

In [18]:
vectors_SVD = makeCV_SVD(vocabulary,sentences,window=2, size=100)

In [19]:
most_similar('Garten',vectors_SVD,10)

[(0.7968530631636896, 'Saal'),
 (0.7582639181295726, 'Grundstück'),
 (0.7424050132867543, 'Wohnhaus'),
 (0.7353572526125755, 'Brunnen'),
 (0.7216225510011339, 'Schlosspark'),
 (0.7198412713285981, 'Ensemble'),
 (0.7187510254743119, 'Tempel'),
 (0.7185339902689601, 'Hof'),
 (0.7173707863339396, 'Gutshof'),
 (0.712802096863283, 'Haus')]

In [20]:
most_similar('betrachten',vectors_SVD,10)

[(0.8120450912559876, 'denken'),
 (0.7604426711698004, 'verstehen'),
 (0.7580560782124063, 'erklären'),
 (0.7558922521025, 'erinnern'),
 (0.7553404684758059, 'bezeichnen'),
 (0.7493517804300217, 'stimmen'),
 (0.745686725644376, 'wenden'),
 (0.7430857644439924, 'bewerten'),
 (0.7419376213951389, 'sprechen'),
 (0.7412548011956499, 'glauben')]

In [21]:
most_similar('Schweden',vectors_SVD,10)

[(0.9090983800625678, 'Ungarn'),
 (0.8903644566846127, 'Russland'),
 (0.8723431524686711, 'Frankreich'),
 (0.8720592656829781, 'Finnland'),
 (0.8718438934345336, 'Norwegen'),
 (0.8643964845014638, 'Dänemark'),
 (0.8633917305898804, 'Polen'),
 (0.8597463453508998, 'Italien'),
 (0.8597153905605144, 'Brasilien'),
 (0.8594229471384974, 'Rumänien')]

In [22]:
most_similar('Park',vectors_SVD,10)

[(0.7875690157831058, 'Forest'),
 (0.7860475212820364, 'Bay'),
 (0.7808347290105225, 'Castle'),
 (0.7788490377496595, 'Airport'),
 (0.7613323497522909, 'Valley'),
 (0.7611110834942387, 'Manhattan'),
 (0.756354978334809, 'Bridge'),
 (0.7486762668134073, 'Avenue'),
 (0.7454323883708169, 'Street'),
 (0.7431295213984858, 'Central')]

We notice that park is not interpreted as a synonym for garden, but rather is perceived as part of English place names. The context 'English words' has apparently become more important.

In [23]:
evaluate(testdata,vectors_SVD)

0.5595562576911485

The optimization was apparently successful:  the correlation has become significantly larger. Bullinaria and Levy (2007), Mohammad and Hirst (2012), Bullinaria and Levy (2012) and Kiela and Clark (2014) provide overviews of the combinations of parameters, training quantities, etc. that lead to optimal results.

## Literature

Rubenstein, H., & Goodenough, J. B. (1965). Contextual Correlates of Synonymy. _Commun. ACM_, 8(10), 627–633. 

Ruge, G., & Schwarz, C. (1990). Linguistically based term associations—A new semantic component for a hyperterm system. _Proceedings of 1st International ISKO-Confercence_, Darmstadt, 88-96.

Crouch, C. J. (1990). An approach to the automatic construction of global thesauri. _Information Processing & Management_, 5, 629–640.

Crouch, C. J., & Yang, B. (1992). Experiments in automatic statistical thesaurus construction. In _Proceedings of the 15th annual international ACM SIGIR conference on Research and development in information retrieval_ (pp. 77-88). ACM.

Grefenstette, G. (1992). Use of syntactic context to produce term association lists for text  retrieval. 89–97.

Karlgren, J., & Sahlgren, M. (2001). From Words to Understanding. In _Foundations of Real-World Intelligence (S. 294–308)_. CSLI Publications.

Bullinaria, J. A., & Levy, J. P. (2007). Extracting semantic representations from word co-occurrence statistics: A computational study. _Behavior research methods_, 39(3), 510-526.

Mohammad, S. M., & Hirst, G. (2012). Distributional measures of semantic distance: A survey. _arXiv preprint_ arXiv:1203.1858.

Bullinaria, J. A., & Levy, J. P. (2012). Extracting Semantic Representations from Word Co-occurrence Statistics: Stop-lists, Stemming and SVD. _Behaviour Research Methods_, 44(3), 890–907.

Mikolov, T., Sutskever, I., Chen, K., Corrado, G. S., & Dean, J. (2013). Distributed representations of words and phrases and their compositionality. In _Advances in neural information processing systems_ (pp. 3111-3119).

Kiela, D., & Clark, S. (2014). A Systematic Study of Semantic Vector Space Model Parameters. In _2nd Workshop on Continuous Vector Space Models and their Compositionality (CVSC)_.

Levy, O., Goldberg, Y., & Dagan, I. (2015). Improving distributional similarity with lessons learned from word embeddings. _Transactions of the Association for Computational Linguistics_, 3, 211-225.